Place de marché
==============

![logo](../reports/figures/logo.png)


### Votre mission
Votre mission est de **réaliser une première étude de faisabilité d'un moteur de classification** d'articles basé sur une image et une description pour l'automatisation de l'attribution de la catégorie de l'article.

Pour ce faire, vous allez **évaluer la possibilité d'extraire des données depuis l'API Amazon** en **prenant connaissance de la documentation** et en **écrivant la requête** qui vous permettrait d'extraire des données supplémentaires. Vous vous assurerez ainsi que vous pourrez bien disposer de plus de données et diversifier les sources de données pour éviter les biais pour votre moteur de classification.

Ensuite, vous **analyserez le jeu de données** déjà constitué en **réalisant un prétraitement** des images et des descriptions des produits, une **réduction de dimension**, puis un **clustering**. Les résultats du clustering seront présentés sous la forme d’une représentation en deux dimensions à déterminer, qui ’illustrera le fait que les caractéristiques extraites permettent de regrouper des produits de même catégorie.

La représentation graphique vous aidera à convaincre Linda que cette approche de modélisation permettra bien de regrouper des produits de même catégorie.

### Contraintes

Linda vous a communiqué les contraintes suivantes :

   * Limiter le nombre d’articles pris par l’API (par exemple : 1000 lignes) et filtrer sur un unique type d’article (par exemple un type d’article peu présent dans votre échantillon de données actuelles).
   * Afin d’extraire les features, mettre en œuvre a minima un algorithme de type SIFT / ORB / SURF.
   * Un algorithme de type CNN Transfer Learning peut éventuellement être utilisé en complément, s’il peut apporter un éclairage supplémentaire à la démonstration.

In [ ]:
import os
import random

import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageOps, ImageFilter

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import plotly.express as px


sns.set(font_scale=1.6)

Image.MAX_IMAGE_PIXELS = 93680329


In [ ]:
def histogram(image, figsize=(12, 8), kde=False):
    image = np.array(image)
    if len(image.shape) > 2:
        # RGB mode
        fig, axes = plt.subplots(3, 1, figsize=figsize)
        for channel, color, ax in  zip(range(3), ['r', 'g', 'b'], axes):
            sns.distplot(image[:, :, channel].flatten(), 
                         kde=kde, color=color, ax=ax)
    else:
        # Gray
        sns.distplot(image.flatten(), kde=kde)

## Chargement des données

In [ ]:
for dirname, _, filenames in os.walk('../data/raw/'):
    if len(filenames) == 1:
        df = pd.read_csv(os.path.join(dirname, filenames[0]))

df['path'] = df['image'].apply(lambda x: os.path.join('../data/raw/Images/', x))

In [ ]:
def extract_level(tree_str, level=-1, strict=False):
    """return a specific level from product_category_tree.
    tips: specify a negative index to access latest part of the tree.
    """
    tree_str = eval(tree_str)[0]
    levels = tree_str.split('>>')
    levels = list(map(lambda x: x.strip(), levels))
    if strict:
        return levels[level]
    else:
        try:
            return levels[level]
        except IndexError:
            return None

In [ ]:
df['label'] = df['product_category_tree'].apply(extract_level, level=1)

In [ ]:
images_ = [x for x in os.listdir('../data/raw/Images/') if x.endswith('.jpg')]
images_ = [os.path.join('../data/raw/Images/', x) for x in images_]

## Reduction dimensionnelle

### niveau de gris

In [ ]:
images = [Image.open(x) for x in images_]
images_gray = [x.convert('L') for x in images]

### resize

In [ ]:
images_128 = [im.resize((128, 128)) for im in images_gray]

In [ ]:
images_128[0]

In [ ]:
images_128[2]

In [ ]:
images[2]

In [ ]:
histogram(images_128[2])

In [ ]:
histogram(images[2], kde=True)

### Transformation sous forme de vecteur

In [ ]:
images_vectors = np.array([np.array(x).reshape((16384)) for x in images_128])
images_vectors.shape

### Analyse en composante principale

In [ ]:
pca = PCA(n_components=50)
pca_50 = pca.fit_transform(images_vectors)

In [ ]:
pca_50.shape

### Projection des données avec T-SNE

In [ ]:
tsne = TSNE(n_components=3)

In [ ]:
tsne_res = tsne.fit_transform(pca_50)

In [ ]:
labels = df['label']
px.scatter_3d(pd.DataFrame({'x': tsne_res[:, 0],
                            'y': tsne_res[:, 1],
                            'z': tsne_res[:, 2], 
                            'label': labels}), 
             x='x', y='y', z='z', color='label')

### Visualisation dans TensorBoard

**warning** Une fois le comportement de TensorFlow v2 désactivé Keras (Voir ResNet50) va lever une exception!


In [ ]:
# import tensorflow.compat.v1 as tf
# from tensorboard.plugins import projector

# tf.disable_v2_behavior()

# tf_data = tf.Variable(pca_50)

# LOG_DIR = '../reports/tf/sessions/pca'

# metadata = 'df_labels.tsv'

# labels.to_csv(os.path.join(LOG_DIR, metadata), index=False)

# with tf.Session() as sess:
#     saver = tf.train.Saver([tf_data])
#     sess.run(tf_data.initializer)
#     saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
#     config = projector.ProjectorConfig()
    
#     embedding = config.embeddings.add()
#     embedding.tensor_name = tf_data.name
    
#     embedding.metadata_path = metadata

#     projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), 
#                                    config)